# **Segement 1: Modules and Model Import**

In [24]:
%pip install -q mauve-text
import os
import torch
import tiktoken
import math
import torch.nn as nn
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
import numpy as np
import mauve

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [25]:

model_path = "/kaggle/input/storygenfinal/pytorch/final/1/model.pth"

#Define the Model Path above ans skip to segment 6 for Generation and Output

# **Segement 2: Torch Device Initialization**

In [26]:
tokenizer = tiktoken.get_encoding('gpt2')
vocab_size = tokenizer.n_vocab
device = "cpu"
if torch.cuda.device_count() > 1:
    device = torch.device("cuda")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **Segement 3: Hyper Parameters Initialization**

In [27]:
train_batch_size = 16
eval_batch_size = 8
context_length = 512
train_split = 0.8
d_model = 512 
n_heads = 8
n_layers = 4
lr = 1e-3
epochs = 10000
eval_steps = 500
max_patience = 1500

# **Segement 4: Model Defination**
Contains
- MultiHead Attention
- Positional Encoding
- Transfomer Block
- Our GPT Like custom Model
- Generate Function


In [28]:
# Multi-head Attention Layer
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, n_heads: int):
        super().__init__()
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads
        assert d_model % n_heads == 0, "d_model must be divisible by n_heads"

        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        self.fc_out = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x: torch.Tensor):
        B, T, D = x.shape

        Q = self.query(x).view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        K = self.key(x).view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        V = self.value(x).view(B, T, self.n_heads, self.head_dim).transpose(1, 2)

        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)

        mask = torch.triu(torch.ones(T, T, device=x.device), diagonal=1).bool()
        attn_scores = attn_scores.masked_fill(mask, float('-inf'))

        attn_weights = torch.softmax(attn_scores, dim=-1)
        attn_output = torch.matmul(self.dropout(attn_weights), V)

        out = attn_output.transpose(1, 2).contiguous().view(B, T, D)
        return self.fc_out(out)

# Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, context_length: int, d_model: int):
        super().__init__()
        pe = torch.zeros(context_length, d_model)
        position = torch.arange(0, context_length).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x: torch.Tensor):
        return x + self.pe[:, :x.size(1)]

# Transformer Block
class GPTBlock(nn.Module):
    def __init__(self, d_model: int, n_heads: int):
        super().__init__()
        self.att = MultiHeadAttention(d_model, n_heads)
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.GELU(),
            nn.Linear(4 * d_model, d_model)
        )
        self.dropout = nn.Dropout(0.2)

    def forward(self, x: torch.Tensor):
        x = x + self.att(self.ln1(x))
        x = x + self.ffn(self.ln2(x))
        return self.dropout(x)

# GPT Model
class GPT(nn.Module):
    def __init__(self, vocab_size: int, context_length: int, d_model: int, n_heads: int, n_layers: int):
        super().__init__()
        self.context_length = context_length
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_enc = PositionalEncoding(context_length, d_model)
        self.blocks = nn.ModuleList([GPTBlock(d_model, n_heads) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, vocab_size, bias=False)

        # Weight tying
        self.head.weight = self.token_emb.weight

    def forward(self, x: torch.Tensor, targets: torch.Tensor = None):
        B, T = x.shape
        assert T <= self.context_length, "Sequence length exceeds context length"

        x = self.token_emb(x)
        x = self.pos_enc(x)

        for block in self.blocks:
            x = block(x)

        x = self.ln_f(x)
        logits = self.head(x)

        loss = None
        if targets is not None:
            logits = logits.view(-1, logits.size(-1))
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, input_ids: torch.Tensor, max_new_tokens: int, temperature: float = 1.0, top_k: int = 0):
        self.eval()
        generated = input_ids

        for _ in range(max_new_tokens):
            input_trunc = generated[:, -self.context_length:]
            logits, _ = self(input_trunc)
            next_token_logits = logits[:, -1, :] / temperature
    
            if top_k > 0:
                top_k_vals, top_k_idx = torch.topk(next_token_logits, k=top_k, dim=-1)
                probs = F.softmax(top_k_vals, dim=-1)
                next_token = top_k_idx.gather(1, torch.multinomial(probs, num_samples=1))
            else:
                probs = F.softmax(next_token_logits, dim=-1)
                next_token = torch.multinomial(probs, num_samples=1)
    
            generated = torch.cat([generated, next_token], dim=1)
    
        return [tokenizer.decode(seq.tolist()) for seq in generated]

        
    def decode(ids):
        return [id2token.get(i, "<unk>") for i in ids if i not in {pad_token_id, eos_token_id}]

m = GPT(
    vocab_size=vocab_size,
    context_length=context_length,
    d_model=d_model,
    n_heads=n_heads,
    n_layers=n_layers
)

if torch.cuda.device_count() > 1:
    m = nn.DataParallel(m)

m = m.to(device)

print(f"Total Parameters: {round(sum(p.numel() for p in m.parameters() if p.requires_grad) / 1_000_000)}M")

Total Parameters: 38M


# **Segement 5: Model Loader**

In [29]:
m.load_state_dict(torch.load(model_path, map_location="cpu"))
m.eval()
mauve_result = mauve.compute_mauve(p_text=["p_generated_text"], q_text=["q_generated_text"], device_id=0)

/tmp/ipykernel_31/1755452276.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  m.load_state_dict(torch.load(model_path, map_location="cpu"))


Featurizing p:   0%|          | 0/1 [00:00<?, ?it/s]

Featurizing q:   0%|          | 0/1 [00:00<?, ?it/s]

WARNING clustering 2 points to 2 centroids: please provide at least 78 training points


In [66]:
#These texts are generated using chatgpt to evaluate our models generation capabilities using mauve score

#MAUVE Evaluation Strings for "the happy prince "
var1 = [
        "The Happy Prince \nA WOLF was just outside the town, who was informed by a beautiful Prince. The latter was saying, 'It is!' said he, 'you understand the one.' And when he caught sight of the sound of kingdom was a strings of precious stones, O paddled in all these Christian lives, with all his mistake, perfect choice in his wrath, but him had only a hard house's. 'His god is in the direction that I can almost be ashamed of that; so you have a third misfortune is in fact,' said the Prince's purpose, 'but I'll ask for it overboard, for we shall have it before I can't come to please myself; and if you do not know what to do. There is not one who has expected tos Mrs. Epanchin's fancy it is too good to stand you down at once.'",    
        "The Happy Prince \nA HTHuments). The Colonel, on the palace of the third day as he tore the future winter with a whiskers by a cloud of swelling— If he was almost out Theseus and preferred to death to bear the Princess. Like his time she wished to care for it. I should like to ask for every house in safety, and she was to be proved to be anxious to him that she liked to insult her. Her cousin lamp therefore she hoped she might have any other state of her husband, and view she had to talk about the town in which she knew as far away. She was full of luxury, but she was just going to ask a task to her. Indeed, instead of it she who said to herself, and did not know what herself was patches of her, and broke. Just then she left the palace of the Queen's palace, and was so beautiful that she could not explain how walking accompanied her as wrong. I was very sorry",    
        "The Happy Prince \nA Cinthya Sarat was not in a episode at an instant their year's secret style; for each look upon his nest in the wind, which the man was to gain, though the dwarfs were white and sharpened along the grass at one side, and then the sun far past. At last he came to the palace to breathe, where the three Hills, where the Prince could not wait long sleep; but the Prince could not carry it in, therefore, been used without what he had left, and at the top of the bird. Her mother was obliged to do this. The peasant took his wife as soon as she feared to know him: 'My dear King,' thought he to himself, 'has no older,' and waited until it was not a no longer, and token of acknowledgment of taking Edward to the glass. 'That is a common Princess,' said Pinocchio, pointing to his head, and pretending to be dumb",    
        "The Happy Prince \nA H marching into the goal of the fair band raised in laughter, more slow than ever on the blast, and condemned by a basket of flame, of the Princess in his private way. The Queen left him, and the King raised the reason; and the Princess to assist him to keep her; frequently whenever he had no strength from the town, he asked him if he would come together. 'It is true,' she said. 'I suppose I am a gardener than it has been, and my father is quite better.' Loki said nothing, and wished to see the danger he gave him the ingratdj at the Queen saw them what was exceedingly foolishly. She told him her to say that she must depend on her driver running after her; and he asked her does not come in by her master. She was so hungry and gaily, and the Queen brought some sisters. But she could not think of it. She rode pale with her soul, and he",        
        "The Happy Prince \nA Riter to the Prince who was about to play with him, but he was a youth at least he determined to remember them AGE, but would not think it would have been responsible for the first time. The chance recognized him, but the Prince was out of the King's palace, who was ordered, and the Emerald City, she gazed at him in this variety of his dove, and found her e'? he found. He remembered that he was awfully glad to have them when she looked at him he was very tired. She wanted to see him alone, would open it. This, however, told him that he thought he would be released from the news that he should save him, and she had been trying to come to his palace, and she sold him, so she would not fall in any placeingly. So she related to her father and my palace she heard allin' glassist. At least, she took a whole house and touched him to the seven Queens, and",
        "The Happy Prince \nIn the way he felt quite high, and he immediately passed his sovereign. The hall was so delighted to be called him that he had put him off, and went out on his back, leaving the more than be heard from a very he wanted to get out to sleep and see them, and stood in its great sorrow into his mouth, and he put his heels in the woods indistinctively, weeping and said: 'Everything cannot speak to you.' Lebuck'Oh!' said he. 'My situation, and it is a most events.' A storm has a strong man, a human friend! And a speech is all of the other happiness who makes his openness in the Fairy's son, and a friend who is a princess.' At this moment the Prince was determined to take his picture glittered in four wheel, and passing by a beautiful Princess painfully for her, to the dishes-box. \u200b",    
        "The Happy Prince \nA COUof songs of seven night. He dressed himself as he made himself one. He tells him to leave a certain result; his paws gently into the doll he set out again, holding it into his hand once more. 'I am gone, I am sorry that I can live twice as well as my sister,' said he, 'you must not die for me to see me, God bless you.' The Prince turned it with a golden crown of motion: it came a little cross; so she told him that he was very much pleased to find her, and past the stable,!go! You are one day, and the young Prince is no use to be promoting that you would never take it on it! Take care to see what you were to save me this state of stories, and the. When he wandered down, the fox called out: 'O thought that I was the satire I never saw myself just enough;t",    
        "The Happy Prince \n'You are with so pleasant?' answered the Prince. 'An' 'And we must,' he said. 'It's the womanriches they died,' said the old woman, 'and there's not a man, and the Princess on every side. The driver had lit at Erisich' 'fill's mother me on your back in the least,' murmured the Prince. Now the Prince caused all his Princess, and her attention. 'It's all red as delightful as you are that you are following them just what they can do to Master Maloney, and he'd leave you a injury.' 'But there are an island there.' 'How pretty it isn't enlightened?' asked the witch. 'I runistle, and if you don't let me up the sound so long as it is here, for it's that's true already.'",    
        "The Happy Prince \nThis meeting on the windows of the exact thing happened, but he had just fitted and husband that which he should have told him. He from those birds and her mother dared do was a clock in the world. Their sons are of impatience, but now whatever be pleasant to doubt 'But it must be true before them.' 'I am my mother,' she answered, 'and thanks to you who own sisterhood.' 'While he had thought he is crying, he accompanied him. I am sure she is more beautiful than she is broken enough 'Sireled to set forth with her.' 'I can't look a good ring in the cloth.' 'Good-by, son,' FredLittle Fairy Brown?' 'I am all about it,' the dear Prince replied. 'And then I am afraid it makes my soul. Has though I am doing me on that morning, or we have him.' 'And who I am a Prince to watch for",    
        "The Happy Prince \nA Liverpool a time before war bore in doors, but the same day was so loon be enrolled by a little girl of her Christ- accelerated that the Wicked Princess, who, as she, was wondering what she could not help seeing them so much as the Princess on account of the Princess, gladly she found himself so well aware of it. 'Acriasing her,' she said to the old dwarf. 'This is for me to take leave her.' And the Princess again passed into her palace, which was connected with every one of her brother subdued waste of fine children was so handsome and different. Some lived as used to have her company, the Princess said, the Fairy, 'What was the Prince? He is offended after her. The King left her and left her, who send her in the two at once, and if she could not be sent. The King and her mother be taken by the anger of the angel. But when she window spoke to her she",
        "The Happy Prince \nALER raised his driver’s basket with a statue beneath them with a girl of straw, turned it from the little cottage and she turned it up in a little piece of beasts, and still his paper. A guest went out with a large boy blowing over it to the youth, slowly, as if so, it seemed to him to be allowed to have killed her with him, saying, full of marrying him as his arm to the Princess. He lost the Prince, gathered up the old woman, pies as he spoke, and saw that she should go out with him, and her head forward; and the Raja put it in her cheeks, who sat on the spot, and so as she drew it and ran to her as she were a deacon with her perch on it like a song; in her castle stood a Princess doing it with the Prince home. Her holy itself was waiting for her tumbled moving down in front of him. He promised him",    
        "The Happy Prince \nthe cat died to carry the pot of the two brothers of the Prince telling and the little bed and golden apples to the Prince’s house which led till she set down for them to the Sultan’s father, who had passed, 'If you were atacity seems to confess that Kingsand is sure so much such as I am, and the Prince is so prominent that you will be obliged to please you that you can drive at ten hundred aonom; I have never ceased to want to have their Royal Highness; and had a fair sermon you will see you a present of the railways in the night, which will allow her to make you safe in our strong. You have obtained less need, she will stay there. Did I not come here by you than go on?' Mr. Buller replied, in a low voice: 'I believe she shall be rumbling beside her, and she will bring me and do it to me these dead ties you all",    
        "The Happy Prince \nJ RamISED. When he first began to toy, sour rows of blood upon the edge of a newspapers forge in a rock which Jack with his Crab turned it over his back, for it was so short that it rose so large that it ended with diamonds. His astonishment, deeply loved attentive to bring spell but he was much more alarmed for his mind to say nothing. He at once looked again: 'I don’t see where you live in this man, if he’s going to drink what we can happen if he’s doing we’ll touch them off.' 'Don’t bear it and look as if he only brought up the ends of him,' said the first moment; and he told his father a few days afterwards, and betrayed his big sloping in the water, and with such a Emperor-place familiar misfortune. They bade them box-bees, who had heard it",    
        "The Happy Prince \nIt trembled at people who thought we should have thought he would have been when they said something to put him in different parts, so that he should be gone. 'I didn't mind where it is,' said Snow-white. 'Why, it is so skilful,' said Ozana; and the master smiled for a moment, and the next day the dawned fellow, she said to her great fear and sat down, and told what U Babam. Then the faintly went high up, turned swiftly into the room again and looked at the outsider, carrying on her. 'And why say he passed away and getting on so with Nihilistications? All the collar was. Her brother put their little pine-flesh, to telling her of the beggar's dress, scales, and she felt run down to the earth. 'I shall die, my brother and bleat,' he said, waving her eyes slowly, 'but I shall not",
    ]


#MAUVE Evaluation Strings for "The Devoted Friend "
var2 = [
    "The Devoted Friend \nHe notions of Rena's statement that he had something very much on a third day, and placed herself in terror; he had played very much of the Cap of 'Little Ozma's lifelong law in such things with her arrival, and was content to give her the speech of his master's driver, but the Carrolicked her high hearts were entirely heavy with wonder; and bringing, in each hour, taking her little room, an arid gentlemanly for something in the world, and such a sudden while they rejoiced that she could well be hoped to see the third. The parents looked dreadful and greatly pleased her much as she feared to give her a dream in her hands, but the Colonel had become so glad that he had not such Christian mind. The large coach did not know how to receive some earthly to ask him that he had never felt almost desirous in its excitement; he supported her for a little day the Mimicops when she had lived just as Queen Lur",
    "The Devoted Friend \nA Finn said to him in a great intervention of theater as she could to be to go down. And she received the conversation to live. The lion gave up her for some time A Brede and the prisoners ferulegroom, who was eventually delirious, he cast upon her. She was a present who happened to do nothing more. She was never to think of the world, and two stood next. And the Great God experienced deceiving that, however, who felt that she could not get in to the palace. She also hesitated, in the same direction, she led her about a little bells. Said Dorothy, 'I know what make of them when they carried them home till their sidelong a King, she said to her minister, 'I will bring you well when they have an much. ar Frederic inappropriate a day, asheart replied, 'Looking-people are as little birds as all they can.' But the Tin Wood,",
    "The Devoted Friend \nYears were called me to risk his ideas and glove for more than ever. Thus being pegged and unusual indefinite as his wife moved. On the littleigh had done? Mrs. Scobell looked 'Who are you?' (I had another creep into bed-fellow; and when he was not in his dinner. He had seen him, and that he was not life yet saying: 'If you go to the parents it is so generous; and let us come near me, you will know how to say it.' He tried to go. He placed it on, but would not go away the rest of her, and forgetting it was impossible for him to go away; and after he had got food and came down to work again with her. She wiped it, it went to the shining stiffest syllables; her heart was although no doubt she could go on, the King had felt his pestilocks in his sleep, and",
    "The Devoted Friend \nTHEThis was distinguishes a little bride's nerves. She adorned her neck, and looked at it this true dumb woman in the sky. Suppose her straw cheeks grew near through her green grass, and her eyes grew splendid and lovely, and asked, 'You must not find her?' 'No, I cannot shudder!' out the Scarecrow. 'If you don't?' shouted the slender, the Wizard spent his father. 'I come to you when you have taken her home, 'that you have lost her.' 'How girl she can do.' 'My dear!' asked the Sultan. 'Well, can show you.' 'I'm glad when she sat with a great feast!' cried the East Street. The Tsar, who wasn't ready,' said Dorothy. 'Oh, my dear,' remarked the man, and the four Abner opened the little girl from both his mouth, and she pushed the Wizard to the old woman and the",
    "The Devoted Friend \nTHE WOL Marcus III The Emerald City was the school. Then the eagle got on the outside seas and sat on fire, and stood without saying two, 'This, you must like her, by Colia,' his wife replied. 'A little girl, a very sweet woman, should be gone.' 'Gentlemen, my child,' said Beauty. 'Oh, you little maiden, I really have already given to the table as you be married.' 'Em, indeed! I wonder about children,' said the girl. 'I have come here!' she said, 'you have better to leave her as my heart's dearest; and I am so glad you must save her. The other women do not like dishonour when she was anxious to see her father. But very soon you could never come to London; for the old woman shall find their turn back, and my father only suits us to find out how to say your mother? I shall still", 
    "The Devoted Friend \n'Speak have the clever course of it,' said the young gentleman who went on again. 'Truly I need remain for you,' he answered, scalyly. 'Why, I don't've got such an hasty room, but your friends can I tell you why, what did you call wested?' 'Here you do my best to do?' said the shadow. 'No, no,' answered the little man. 'Yes, I'd like to find a judge. How happy you are, very sorry you've been going into dinner?' he added, 'but you know. Pray put it in not, but it is.' 'Oh! why my mother, Though you thought she was goin' fair, so I'm so fondin' 'em in my eyes a bit for it.' 'I've got hold of it, Mark,' said the man. He bowed from his hat and gave him a little",
    "The Devoted Friend \nThere were far below for half an hour weaker old lady who stood by Free-or by it. For of Famine with a mother that was much to meet “to go out by your illness to the faithful recognain mortally, which, of which in its command in Otter letters and white childhood were made of the Snow Queen who asked her how exactly like the other people, but who was too beautiful to see the witch, and became a fierce one, absent child, with a fairyland of a young lady exactly like whom luxury, which offended the Mimics should make them unusual this to go. The Queen, however, did not advise her to get no fault at all. The Queen seemed to have rejoicing, and was likewise to be seen through to seek her. She sent her. The Queen, who was in explanation; and as therefore by him she could, she sat in the narrow room, and did not look upon her. By means she had remained able to",    
    "The Devoted Friend \nA WALA EARAGE LE T EDforth falls off in a field. So the plum-continued end of the Grand Daedalous and precautions who had a weather-shop. They had minutes, however, in order to spoil their own into the full hall at the end; for in question that they must still force them just as all they had happened, they leaned back of the whole court. When the Sultan conveyed, Quoh, they knew at once how damp they had tried to teach them to put Dundeeching them were sitting at mountains to the palace in countries; which arose there were no other single dog, but as the little Fluker, there was a blue spellodington to raise them at the nursery, for at the cry. As they met the gardener's great king; the latter, however, they had a forehead on the ground, the magician argued they had scarcely looked before him. Having got the wonders the",    
    "The Devoted Friend \n   And the young schools, Simeon laughed, for brideghed, and then whispered, 'Why, a couple of years,' said the youth, and the former hoped well, and assured her what her son and her mother been. 'Would you not try to be getting on a mighty lady?' said the mother, 'I shall not change your mother's whole heart the husband! I may satisfy you.' The little girls thought they had never heard of her at all, for or nothing it would be everywhere to theued-room. The mother was alarmed, and said that the Sultan had bank-in-law, but the Ulysses had given them for her. Tuk had been dismal by her that Snow Queen called their children to Edward and Queen Ra told her all that journey. So told the King.' The King and Queen Caroline  The Queen preferred Ulysses A SCHOOL attributed her",
    "The Devoted Friend \nGeorge we deprived Mercury with the red books darker and him. There were Violet and perseverance: they climbed and listened, and they saw Bear behind them, and not again; but she sprang into the thicket that flew in the out of gold, but who was always silent, furtively, and at length, but they took to them and went out immediately to school. 'Oh!' said she to her. 'It is mine, and we have to go to church, and so we must go with us to the page; and as we know that we may have to get the garden, for he is very much summoned to do. I will try to get all to sleep, ins and I have to go to bed. He who has hitherto been able to be forgotten and when we recognize that he loves you.' 'You cannot see everything,' said she; 'I am very pleased:' thought she to herself",    
    "The Devoted Friend \nAccording to the family knew going to go back to the Princess before these words, and to her father, instead of compassion asked what he could say about the gardener's Terrible. The child was still more than two of the Queen, and happens, and she took up the use of her eye to take the men in her hand, he when she had started what she meant, and put on her hand which she dared to take away. She loved to send her way to pick a time to supposing us, and she cried by the Princess earned, so that she begged her to go into the Princess, which drove to her to lay the ball. She was to least lie so long that punctuals. It was as near as her sister and it was too late. encouraged the Princess, who since she sat with her son of delight that she had done respectfully; and as she was, in a very sad garden, and whose mother who had not received a map",    
    "The Devoted Friend \nthose eleven birds wanted bright danced about the most good old people, but congratulated them not long ago, they had obliged such already, they love all. There they ate nothing on their work orileings that could not possibly understand the people; and they had without tormenting and therefore 'fatur!' they began to ask for the mountain the whole night. 'I know that a month since I should have been.' as the old Miller pleased, somehow, 'My anticipations that my trade may be lately; so always whatever I have been about to do so!' 'It is a very quiet way to he; and the shadow of the widow knew that the man was really out of the city, who stood in the cellar every good time, and forgive me on the day: therefore I am tired about it, and I propose to give me a year for my wife to bid you for that, and then he has the Elder Tree.",    
    "The Devoted Friend \nTHE Antaeus stood according to the lady, and she looked about her for a loud voice, where she was! Up to look at her feet. 'I am getting very good! And a little treat you are passing by tomorrow I know which you are,' said she; 'you will make a son.' Half a moment she smiled, 'you will stay here, for you will stir mankind here at the great palace, and I will tell you to make myself, but Mrs. Jane into a little boy who is called her son, and she wants me to marry me. And then come up here and then you shall go to bed with me. If you will be just as well as a man! You must get to it, you shall allow me to go out to the floor, and 'em very well you will find me a fine thing to do.' But the little boy examined him, and said: 'It is",
    "The Devoted Friend \nA CRAER summer passed upon the traveller. It was equally short or Bear himself, but on the Princeseless king's age, who had great tears abroad that had long been sent to fight with them; however, with a strong, she had to touch her hand and I examined the danger for her who it was proud of him to be cold. By this time she got from any questions violently that the Prince had caught her hand, and then took his hand with her first eyes, and she grew out of the home. And there was a mark that had given him the first impression had never seen him, and the King heard rein, but he thought hastily. At last he came to the King to find her, and thought him what she had been refused, and a coachman and liquor, they waited until she was discreet and he fancied that it was the son of the King but had called for him done, when he climbed up to the hill, and this was going",    
    "The Devoted Friend \nA WASTED ted around the thing that had been upon him. discovering this was still most beautiful of such as he was. The dominion of the two ladies of the violetkeepers were very comfortable, but has appeared to be knocked by theplace. SureSpain were the fairies of the bottles; but the little children from their great neighbors were obliged to be time to prove their cracked for beauty. So the old grandmother was very proud of them, and, taking their compositions, Juno sat on one stroke in a chair, and told her that they had been thinking so early in their opinion that they had to go on Trusty John's party of theirergus who had heard their arrival for them, should really have been freed from the queen, and then was to be tempted, and Dashwoods always convulsions, which went into the courtyard."
]

# **Segement 6: Text Generation**

In [69]:
q_generated_text=var1 #adjust according to your prompt 


with torch.no_grad():
    input = torch.tensor(tokenizer.encode("The Happy Prince "), dtype=torch.long, device=device).unsqueeze(0)
    p_generated_text = m.module.generate(input, max_new_tokens=200)[0]
    print(p_generated_text)
    print("\n\n")
    mauve_result = mauve.compute_mauve(
        p_text=[p_generated_text],
        q_text= q_generated_text,
        device_id=0)
    print(f"\n\nMAUVE score: {mauve_result.mauve}")

The Happy Prince 
A MAN took his magic order and the102 gave it to mirth. And the Giant guessed that he could dwell in the instant of his defenses, that he was the Swallow spoke. The Tsar led him to the door of the door that seemed to him quite ignorant that it must be very beautiful to his beloved, but there was no enjoyment. Before he was employed to lend her out, he remembered what, there was no means to do. The Marionette said him, “Dear HER.” So the four, as they lay down the ground to keep him; but they did not lose their conversation, but at last they passed their holes in their cup. “Hither agree to undermine that the winter's mischief I should buy you a clerk,” he added, so he went home to the end of his living company, and that the Prince was elected, and the soldiers and Hans had a whole beautiful situation; but he continued to be very hungry,





Featurizing p:   0%|          | 0/1 [00:00<?, ?it/s]

Featurizing q:   0%|          | 0/14 [00:00<?, ?it/s]



MAUVE score: 0.7255568962531734


WARNING clustering 15 points to 2 centroids: please provide at least 78 training points


**try variations with temperature and top_k**

In [ ]:
with torch.no_grad():
    input = torch.tensor(tokenizer.encode("The Happy Prince "), dtype=torch.long, device=device).unsqueeze(0)

    variations = [
        {"temperature": 0.7, "top_k": 50},
        {"temperature": 1.0, "top_k": 40},
        {"temperature": 1.2, "top_k": 30},
    ]
    
    for i, params in enumerate(variations, 1):

        p_generated_text = m.module.generate(input, max_new_tokens=200, temperature=params["temperature"], top_k=params["top_k"])[0]
        print(f"\n--- Variation {i} (temp={params['temperature']}, top_k={params['top_k']}) ---\n")
        print(f"Text: {p_generated_text}")
